# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath, filemask = '*.*', exclude = ''):
    '''
    Returns the list of files under a given filepath, mask, and excluding specified system subfolders.

    Parameters:
        filepath (str): Path to start with and look into subfolders;
        filemask (str): Files to look at, e.g. "*.json";
        exclude  (str): Subfolder(s) name (or tuple) to exclude from the list.

        Returns:
            all_files (list): A list of files matching the specified conditions.
    '''    
    all_files = []
    for root, dirs, filez in os.walk(filepath): # 'files' looks unsafe here because of the same local variable
        if exclude and root.endswith(exclude):
            files = []
        else:
            files = glob.glob(os.path.join(root, filemask))

        for f in files:
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data', '*.json', exclude = '.ipynb_checkpoints')

In [5]:
filepath = song_files[0]
filepath

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [6]:
df = pd.read_json(filepath, typ = 'series') # https://stackoverflow.com/a/38381219
print(df)

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                   None
artist_longitude                  None
artist_location        California - LA
artist_name                     Casual
song_id             SOMZWCG12A8C13C480
title                 I Didn't Mean To
duration                       218.932
year                                 0
dtype: object


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
#INSERT INTO songs (song_id, title, artist, year, duration)
song_data = (df['song_id'], df['title'], df['artist_id'], df['year'], df['duration'])
song_data

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
#INSERT INTO artists (artist_id, name, location, latitude, longitude)
artist_data = (df['artist_id'], df['artist_name'], df['artist_location'], df['artist_latitude'], df['artist_longitude'])
artist_data

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data', '*.json', exclude = '.ipynb_checkpoints')

In [12]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [13]:
df = pd.read_json(filepath, lines = True) # https://stackoverflow.com/a/41226605
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df.page.eq('NextSong')]
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [15]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = (t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [17]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
# Level is not a permanent attribute of a user, I think it's incorrect to set this dimension based on the log_data.
# I can include timestamp to users_df and sort it descending, so at least we have the latest mentioned level of a user.

#INSERT INTO users (user_id, first_name, last_name, gender, level)
user_df = df[["ts", "userId", "firstName", "lastName", "gender", "level"]].sort_values(by = ["ts"], ascending = False)
user_df.head()

,ts,userId,firstName,lastName,gender,level
387,1543607664796,5,Elijah,Davis,M,free
385,1543603884796,16,Rylan,George,M,paid
384,1543603678796,16,Rylan,George,M,paid
383,1543603476796,16,Rylan,George,M,paid
382,1543603205796,16,Rylan,George,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row[1:]) #[1:] excludes the timestamp
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
# I had to use string for song length since WHERE condition on floating point value proved not reliable to me.

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, str(row.length)))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    #INSERT INTO songplays (songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    #VALUES (DEFAULT, TIMESTAMP %s, %s, %s, %s, %s, %s, %s, %s)
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent.strip('"'))
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

- I'm going to replace the iterative insert with [copy of the dataframe from the memory using StringIO](https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/). Copy insert is further used in `etl.py`.

In [21]:
from io import StringIO

def copy_from_stringio(conn, df, table, which_columns = None):
    '''
    Save the dataframe in memory and use copy_from() to copy it to the table.
    based on https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/

    Parameters:
        conn (connection): From psycopg2.connect();
        df (DataFrame): DataFrame to be copied into the table;
        table (str): Subfolder(s) name (or tuple) to exclude from the list;
        which_columns (iterable): Column names, use to omit serial or defaulting attributes. If not specified, it is assumed that the entire table matches the file structure.
    '''    
    buffer = StringIO()
    df.to_csv(buffer, index = False, header = False, doublequote = False, sep = '\t', na_rep = 'NULL', escapechar = '"')
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep = "\t", null = 'NULL', columns = which_columns)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

songplay_cols = ['start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent']
songplay_df = pd.DataFrame(columns = songplay_cols)

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, str(row.length)))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    songplay_row = [pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent.strip('"')]
    songplay_df = songplay_df.append(dict(zip(songplay_cols, songplay_row)), ignore_index=True)

copy_from_stringio(conn, songplay_df, 'songplays', songplay_cols)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.